In [ ]:
#import packages

import os
import sys
import pandas as pd
import numpy as np
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import igraph as ig

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os
import math
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType 
from graphframes import *

import matplotlib.pyplot as plt
%matplotlib inline



## Import Data

In [ ]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()


In [ ]:
df_followers = pd.read_csv('tweet_retweet_follower_network.csv', dtype={'user_id':str, 'followers':str})
df_followers

,followers,user_id,is_fake_news,follow_or_retweet
0,787311228,43350851,0.0,retweet
1,3338246572,16297707,0.0,retweet
2,754310205546954757,5820642,0.0,retweet
3,2157919340,1716121,0.0,retweet
4,4867759271,31056977,0.0,retweet
...,...,...,...,...
4032402,3306283622,3416678236,NaN,follower
4032403,4102376488,3416678236,NaN,follower
4032404,4165642155,3416678236,NaN,follower
4032405,384708626,3416678236,NaN,follower


In [ ]:
df_retweets = df_followers[df_followers['follow_or_retweet']=='retweet']
df_retweets 

,followers,user_id,is_fake_news,follow_or_retweet
0,787311228,43350851,0.0,retweet
1,3338246572,16297707,0.0,retweet
2,754310205546954757,5820642,0.0,retweet
3,2157919340,1716121,0.0,retweet
4,4867759271,31056977,0.0,retweet
...,...,...,...,...
349139,NaN,2505854384,1.0,retweet
349140,NaN,207579998,1.0,retweet
349141,NaN,471264778,1.0,retweet
349142,NaN,3001447804,1.0,retweet


In [ ]:
#get tweet retweet network

df_tweet_retweet = pd.read_csv('tweet_retweet_network.csv')
df_tweet_retweet

,tweet_id,user_id_retweet_df,user_id_tweet_df,news_id,is_fake_news
0,1033706162695356417,7.873112e+08,4.335085e+07,politifact99,0.0
1,1033706162695356417,7.873112e+08,4.335085e+07,politifact340,0.0
2,1035580865160638464,3.338247e+09,1.629771e+07,politifact99,0.0
3,1035580865160638464,3.338247e+09,1.629771e+07,politifact340,0.0
4,934206237708865537,7.543102e+17,5.820642e+06,politifact99,0.0
...,...,...,...,...,...
573632,813192381583466496,NaN,3.001448e+09,politifact13600,1.0
573633,812814063763918848,NaN,5.256945e+08,politifact13600,1.0
573634,918510122363744257,NaN,1.525687e+09,politifact14621,1.0
573635,917522953390223366,NaN,7.889922e+17,politifact14621,1.0


In [ ]:
# drop na

df_tweet_retweet.dropna(inplace=True)
df_tweet_retweet

,tweet_id,user_id_retweet_df,user_id_tweet_df,news_id,is_fake_news
0,1033706162695356417,7.873112e+08,4.335085e+07,politifact99,0.0
1,1033706162695356417,7.873112e+08,4.335085e+07,politifact340,0.0
2,1035580865160638464,3.338247e+09,1.629771e+07,politifact99,0.0
3,1035580865160638464,3.338247e+09,1.629771e+07,politifact340,0.0
4,934206237708865537,7.543102e+17,5.820642e+06,politifact99,0.0
...,...,...,...,...,...
89022,813052944295686144,1.496571e+09,7.960258e+17,politifact13600,1.0
89023,813052947743395840,9.816488e+08,7.848257e+17,politifact13600,1.0
89024,812625752197775360,7.200451e+08,2.435832e+07,politifact13600,1.0
89025,918082761990303745,8.990504e+08,1.342402e+08,politifact14621,1.0


## All News Network

### Network Creation

In [ ]:
#all news network

df_net_all = df_tweet_retweet[['user_id_retweet_df', 'user_id_tweet_df']]
df_net_all

,user_id_retweet_df,user_id_tweet_df
0,7.873112e+08,4.335085e+07
1,7.873112e+08,4.335085e+07
2,3.338247e+09,1.629771e+07
3,3.338247e+09,1.629771e+07
4,7.543102e+17,5.820642e+06
...,...,...
89022,1.496571e+09,7.960258e+17
89023,9.816488e+08,7.848257e+17
89024,7.200451e+08,2.435832e+07
89025,8.990504e+08,1.342402e+08


In [ ]:
#drop duplicates (since graphframes cannot take weight)

df_net_all = df_net_all.drop_duplicates()
df_net_all

,user_id_retweet_df,user_id_tweet_df
0,7.873112e+08,4.335085e+07
2,3.338247e+09,1.629771e+07
4,7.543102e+17,5.820642e+06
5,2.157919e+09,1.716121e+06
7,4.867759e+09,3.105698e+07
...,...,...
89021,7.092809e+17,6.957517e+17
89022,1.496571e+09,7.960258e+17
89023,9.816488e+08,7.848257e+17
89024,7.200451e+08,2.435832e+07


In [ ]:
# get list of nodes

retweet_net_nodes = list(set(df_net_all['user_id_retweet_df'].tolist())) + list(set(df_net_all['user_id_tweet_df'].tolist()))

In [ ]:
#df_net2.to_csv('tweet_retweet_network_edges.csv', index = False)

In [ ]:
# convert to node and id format for graphframes

df_retweet_net_nodes  = pd.DataFrame({'id':retweet_net_nodes,'node':retweet_net_nodes}).drop_duplicates()
df_retweet_net_nodes 

,id,node
0,7.543102e+17,7.543102e+17
1,9.849655e+17,9.849655e+17
2,1.015009e+18,1.015009e+18
3,7.626140e+17,7.626140e+17
4,7.769040e+17,7.769040e+17
...,...,...
122235,4.443340e+07,4.443340e+07
122236,3.412722e+09,3.412722e+09
122237,2.162688e+07,2.162688e+07
122238,2.768634e+09,2.768634e+09


In [ ]:
#df_retweet_net_nodes.to_csv('retweet_net_nodes.csv', index = False)

In [ ]:
# create spark dataframe for nodes

df_spark_nodes = spark.read.csv("retweet_net_nodes.csv", header = True)
df_spark_nodes.show()

+--------------------+--------------------+
|                  id|                node|
+--------------------+--------------------+
|7.543102055469548...|7.543102055469548...|
|9.849654828716524...|9.849654828716524...|
|1.015008590644510...|1.015008590644510...|
|7.626139826791383...|7.626139826791383...|
|7.769040225485455...|7.769040225485455...|
|8.665793926122701...|8.665793926122701...|
|          46923782.0|          46923782.0|
|8.300756281834865...|8.300756281834865...|
|9.67092784061612e+17|9.67092784061612e+17|
|         133955593.0|         133955593.0|
|         260440068.0|         260440068.0|
|         236716043.0|         236716043.0|
|          17563655.0|          17563655.0|
|         326500365.0|         326500365.0|
|        3279552517.0|        3279552517.0|
|        1294729226.0|        1294729226.0|
|        2576432274.0|        2576432274.0|
|         201457681.0|         201457681.0|
|          19005467.0|          19005467.0|
|        1018560540.0|        10

In [ ]:
# create edge list dataframe for graphframes

df_spark_edges = spark.read.csv("tweet_retweet_network_edges.csv", header = True)
df_spark_edges =  df_spark_edges.selectExpr(
    'user_id_tweet_df AS src',
    'user_id_retweet_df AS dst',
)
df_spark_edges.show()

+------------+--------------------+
|         src|                 dst|
+------------+--------------------+
|  43350851.0|         787311228.0|
|  16297707.0|        3338246572.0|
|   5820642.0|7.543102055469548...|
|   1716121.0|        2157919340.0|
|  31056977.0|        4867759271.0|
|  13115682.0|         370794560.0|
|  22696550.0|1.034479942895951...|
| 135247359.0|         254708177.0|
|  16312576.0|          27128191.0|
|  17112878.0|        1325337348.0|
|  10400902.0|        1924285831.0|
| 703904681.0|        2548750220.0|
| 993451813.0|        3107597132.0|
|  50174697.0|        4799127991.0|
| 787969735.0|         479832877.0|
|  15552221.0|         121543274.0|
|  17921386.0|9.77722039845843e+17|
|  18812572.0|          85109187.0|
|2463977196.0|         171973084.0|
|   6134882.0|        4508418136.0|
+------------+--------------------+
only showing top 20 rows



In [ ]:
#create graph

graph = GraphFrame(df_spark_nodes, df_spark_edges)

### Calculate Pagerank

In [ ]:
# get pagerank for all news network

pg_rank= graph.pageRank(resetProbability=0.15, tol=0.01)

In [ ]:
#sort in order of pagerank

pg_rank.vertices.select("id", "pagerank").sort(col('pagerank').desc()).show()

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|          40173650.0|28.587843572283454|
|        4099926287.0|25.932091952476313|
|9.683736282566205...| 24.52030950599248|
|7.158926782656922...|17.986571285780798|
|        4521330855.0|17.532812934676027|
|          44299861.0|15.785445329071587|
|        1151933252.0|13.050845908010466|
|7.32109729472766e+17|12.942240162841294|
|7.530574674965586...|11.678688931218755|
|9.576569107481436...|11.428710240128021|
|7.244819744510812...|10.832734552759383|
|        2194368678.0|10.668841238733007|
|        2347323266.0| 9.374580704330779|
|8.031785258932675...| 9.267801727010564|
|         195030017.0| 9.083474747397862|
| 7.8041626592401e+17| 9.044807490653465|
|        1418900996.0| 9.018279744151016|
|         885529764.0| 8.922301377138801|
|8.236770814226883...|  8.81862496174266|
|        2713556066.0| 8.770128010544033|
+--------------------+------------

## Fake News Network

### Network Creation

In [ ]:
# Fake news network

df_tweet_retweet_fake = df_tweet_retweet[df_tweet_retweet['is_fake_news'] == 1.0]
df_tweet_retweet_fake

,tweet_id,user_id_retweet_df,user_id_tweet_df,news_id,is_fake_news
4125,1019236802639646721,8.856920e+17,3.974499e+07,politifact14667,1.0
4161,1019211124628221957,1.917218e+09,8.277065e+07,politifact14667,1.0
4235,1019419664395497473,2.559283e+08,2.147477e+07,politifact14667,1.0
4329,1019225151865581568,8.222716e+17,8.247972e+17,politifact14667,1.0
4379,1019453036106678273,1.759702e+07,1.109180e+08,politifact14667,1.0
...,...,...,...,...,...
89022,813052944295686144,1.496571e+09,7.960258e+17,politifact13600,1.0
89023,813052947743395840,9.816488e+08,7.848257e+17,politifact13600,1.0
89024,812625752197775360,7.200451e+08,2.435832e+07,politifact13600,1.0
89025,918082761990303745,8.990504e+08,1.342402e+08,politifact14621,1.0


In [ ]:
#get relevant columns

df_net_fake = df_tweet_retweet_fake[['user_id_retweet_df', 'user_id_tweet_df']]
df_net_fake

,user_id_retweet_df,user_id_tweet_df
4125,8.856920e+17,3.974499e+07
4161,1.917218e+09,8.277065e+07
4235,2.559283e+08,2.147477e+07
4329,8.222716e+17,8.247972e+17
4379,1.759702e+07,1.109180e+08
...,...,...
89022,1.496571e+09,7.960258e+17
89023,9.816488e+08,7.848257e+17
89024,7.200451e+08,2.435832e+07
89025,8.990504e+08,1.342402e+08


In [ ]:
#drop duplicates

df_net_fake = df_net_fake.drop_duplicates()
df_net_fake

,user_id_retweet_df,user_id_tweet_df
4125,8.856920e+17,3.974499e+07
4161,1.917218e+09,8.277065e+07
4235,2.559283e+08,2.147477e+07
4329,8.222716e+17,8.247972e+17
4379,1.759702e+07,1.109180e+08
...,...,...
89021,7.092809e+17,6.957517e+17
89022,1.496571e+09,7.960258e+17
89023,9.816488e+08,7.848257e+17
89024,7.200451e+08,2.435832e+07


In [ ]:
#create node list

retweet_net_fake_nodes = list(set(df_net_fake['user_id_retweet_df'].tolist())) + list(set(df_net_fake['user_id_tweet_df'].tolist()))

In [ ]:
#df_net_fake.to_csv('fake_tweet_retweet_network_edges.csv', index = False)

In [ ]:
# create node dataframe in correct format for graphframes

df_fake_retweet_net_nodes  = pd.DataFrame({'id':retweet_net_fake_nodes,'node':retweet_net_fake_nodes}).drop_duplicates()
df_fake_retweet_net_nodes 

,id,node
0,1.017952e+18,1.017952e+18
1,8.963754e+17,8.963754e+17
2,9.331858e+17,9.331858e+17
3,7.012540e+17,7.012540e+17
4,3.020456e+09,3.020456e+09
...,...,...
32307,2.801991e+08,2.801991e+08
32308,4.295883e+07,4.295883e+07
32309,1.167393e+09,1.167393e+09
32310,2.191884e+09,2.191884e+09


In [ ]:
#df_fake_retweet_net_nodes.to_csv('fake_retweet_net_nodes.csv', index = False)

In [ ]:
# spark node dataframe for fake news

df_spark_nodes_fake = spark.read.csv("fake_retweet_net_nodes.csv", header = True)
df_spark_nodes_fake.show()

+--------------------+--------------------+
|                  id|                node|
+--------------------+--------------------+
|1.017951685224759...|1.017951685224759...|
|8.963754485394964...|8.963754485394964...|
|9.331857980227584...|9.331857980227584...|
|7.012539992283546...|7.012539992283546...|
|        3020455940.0|        3020455940.0|
|8.431721255445135...|8.431721255445135...|
|8.51566275251798e+17|8.51566275251798e+17|
|9.010300909806879...|9.010300909806879...|
|1.050809902397096...|1.050809902397096...|
|9.081092457779364...|9.081092457779364...|
|        3524001795.0|        3524001795.0|
|          38109190.0|          38109190.0|
|        1278904513.0|        1278904513.0|
|        2576432274.0|        2576432274.0|
|         205488145.0|         205488145.0|
|        3609952274.0|        3609952274.0|
|        1497373045.0|        1497373045.0|
|          28377110.0|          28377110.0|
|        1077444631.0|        1077444631.0|
|        3227484185.0|        32

In [ ]:
# edge list for fake news

df_spark_edges_fake = spark.read.csv("fake_tweet_retweet_network_edges.csv", header = True)
df_spark_edges_fake =  df_spark_edges_fake.selectExpr(
    'user_id_tweet_df AS src',
    'user_id_retweet_df AS dst',
)
df_spark_edges_fake.show()

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|          39744993.0|8.856920025910067...|
|          82770649.0|        1917218150.0|
|          21474769.0|         255928301.0|
|8.247972124251914...|8.222716323832955...|
|         110918004.0|          17597022.0|
|         184929716.0|          22824243.0|
|         110918004.0|        1378552322.0|
|         191589342.0|         188343111.0|
|         110918004.0|1.017951685224759...|
|        4520399652.0|           9924742.0|
|         110918004.0|         915331472.0|
|8.666906389091041...|9.605719827807314...|
|        2850251396.0|        2716850508.0|
|          62088756.0|9.480252883566428...|
|7.462364706843116...|7.594212343753318...|
|          47299616.0|         103253996.0|
|1.004813571161755...|8.336089029356052...|
|9.321231456657818...|         120725508.0|
|         311552533.0|1.009445927491395...|
|        1323937488.0|        48

In [ ]:
#fake news graphframe

graph_fake = GraphFrame(df_spark_nodes_fake, df_spark_edges_fake)

### Calculate Pagerank

In [ ]:
# calculate pagerank

pg_rank_fake= graph_fake.pageRank(resetProbability=0.15, tol=0.01)

In [ ]:
# show page rank in order

pg_rank_fake.vertices.select("id", "pagerank").sort(col('pagerank').desc()).show()

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|7.32109729472766e+17|10.954869344581533|
|        3960210861.0| 8.769915904421234|
|        1418900996.0| 8.591319746680423|
|7.045636060132147...| 6.372410515955808|
|          51191522.0| 6.150792370803543|
|8.579367281658962...| 5.747850288708513|
|        2293749414.0| 5.330517417967232|
|7.311571039734415...| 4.750568635523314|
|8.236770814226883...| 4.639759562947181|
|        4011800885.0|  4.59358911604046|
|        1736787728.0|4.4861845415418315|
|7.711943646220124...|4.4861845415418315|
|7.158926782656922...| 4.392957537663975|
|7.530574674965586...| 4.337553001375909|
|          19218108.0|  4.16546315381449|
|8.04778581955969e+17| 4.088653345717449|
|9.787638791561912...| 4.065567095961764|
|8.183863368140227...| 4.035346439804637|
|         900431706.0| 4.012680947686791|
|        3589050802.0| 3.934610919280879|
+--------------------+------------

## Analysis on Page Rank

- The above pagerank spark dataframes were exported and analysed with pandas

In [2]:
df_all = pd.read_csv('pagerank_all_news.csv')
df_all

,id,pagerank
0,4.017365e+07,28.587844
1,4.099926e+09,25.932092
2,9.683736e+17,24.520310
3,7.158927e+17,17.986571
4,4.521331e+09,17.532813
...,...,...
118481,4.443340e+07,0.701148
118482,3.412722e+09,0.701148
118483,2.162688e+07,0.701148
118484,2.768634e+09,0.701148


In [3]:
df_fake = pd.read_csv('pagerank_fake_news.csv')
df_fake

,id,pagerank
0,7.321097e+17,10.954869
1,3.960211e+09,8.769916
2,1.418901e+09,8.591320
3,7.045636e+17,6.372411
4,5.119152e+07,6.150792
...,...,...
31504,2.801991e+08,0.711074
31505,4.295883e+07,0.711074
31506,1.167393e+09,0.711074
31507,2.191884e+09,0.711074


In [4]:
df_all['all_news_rank'] =  df_all.pagerank.rank(method='dense',ascending=False).astype(int)
df_all

,id,pagerank,all_news_rank
0,4.017365e+07,28.587844,1
1,4.099926e+09,25.932092,2
2,9.683736e+17,24.520310,3
3,7.158927e+17,17.986571,4
4,4.521331e+09,17.532813,5
...,...,...,...
118481,4.443340e+07,0.701148,3108
118482,3.412722e+09,0.701148,3108
118483,2.162688e+07,0.701148,3108
118484,2.768634e+09,0.701148,3108


In [5]:
df_fake['fake_news_rank'] =  df_fake.pagerank.rank(method='dense',ascending=False).astype(int)
df_fake

,id,pagerank,fake_news_rank
0,7.321097e+17,10.954869,1
1,3.960211e+09,8.769916,2
2,1.418901e+09,8.591320,3
3,7.045636e+17,6.372411,4
4,5.119152e+07,6.150792,5
...,...,...,...
31504,2.801991e+08,0.711074,690
31505,4.295883e+07,0.711074,690
31506,1.167393e+09,0.711074,690
31507,2.191884e+09,0.711074,690


In [14]:
df_both = pd.merge(df_all[['id','all_news_rank']],df_fake[['id','fake_news_rank']], on = 'id')
df_both = df_both.sort_values('fake_news_rank').reset_index(drop = True)
df_both.id = df_both.id.astype(str)
df_both

,id,all_news_rank,fake_news_rank
0,7.32109729472766e+17,8,1
1,3960210861.0,21,2
2,1418900996.0,17,3
3,7.045636060132147e+17,31,4
4,51191522.0,56,5
...,...,...,...
31504,927504176.0,3108,690
31505,405051209.0,3108,690
31506,1915787083.0,3108,690
31507,1.0052665031370506e+18,3108,690
